In [ ]:
from nb_shared import *

import math

In [ ]:
snow17_db = SweDB(SNOW_17_DB_CONNECTION)
zone_infos = CBRFCZone(DB_CONNECTION_INFO)
swann_data = SWANNZonalSWE(DB_CONNECTION_INFO)

## Target Zones

In [ ]:
target_ch5ids = [
    'ANBC2',
    'RCYC2',
    'ALEC2',
    'ENMC2',
    'WBRW4',
    'BPPA3',
    'OAWU1'
]

In [ ]:
target_zones = zone_infos.from_ch5_ids(target_ch5ids)

# Get SWE for each year

## Snow-17 SWE

In [ ]:
snow17_swe = {
    cz.zone: snow17_db.for_zone(segid=cz.ch5_id + 'H', opid=cz.zone) for cz in target_zones
}

## SWANN for CBRFC zone

In [ ]:
swann_swe = {
    cz.zone: swann_data.for_zone(cz.gid) for cz in target_zones
}

In [ ]:
def plot_zone(zones):
    fig, (ax1, ax2) = plt.subplots(2, 1, dpi=300, sharex=True, figsize=(12,5))
    fig.subplots_adjust(hspace=0)
    colors = ['cadetblue', 'goldenrod', 'crimson']

    SWE_col = 'SWE (mm)'
    max_y = 0

    for index, zone in enumerate(zones):
        s17 = snow17_swe[zone].loc['1999-10-01':'2020-07-31']
        swann = swann_swe[zone].loc['1999-10-01':'2020-07-31']
        diff = s17[SWE_col] - swann['swe']
        
        ax1.plot(
            s17.index, s17['SWE (mm)'], label=f'{zone}', color=colors[index], alpha=0.6,
        )
        # ax.plot(
        #     swann.index, swann[zone_name + '_SWANN'], 
        #     label='UA SWE', ls=(0, (5, 1)), lw=0.8, color='goldenrod'
        # )
        ax2.plot(
            s17.index, diff, label=f'{zone}', lw=0.9, color=colors[index], alpha=0.7,
        )

        max_swe = math.ceil(s17[SWE_col].max())
        max_y = max_swe if max_swe > max_y else max_y

    
    ax2.axhline(0, color='black', ls='dashed', lw=0.6)
    bbox = dict(boxstyle="round4,pad=0.3", fc="powderblue", lw=0)
    ax2.annotate('SNOW 17 higher', xy=(0.01, 0.9), fontstyle='italic', xycoords='axes fraction', bbox=bbox)
    bbox = dict(boxstyle="round4,pad=0.3", fc="thistle", lw=0)
    ax2.annotate('UA SWE higher', xy=(0.01, 0.05), fontstyle='italic', xycoords='axes fraction', bbox=bbox)

    ax1.set_ylim(bottom=-25, top=(max_y + 25))
    ax1.set_xlim(left=s17.index[0], right=s17.index[-1])
    
    ax1.set_ylabel(r"SNOW-17 SWE (mm)")
    ax1.legend(loc='upper left', ncols=3, frameon=False)
    
    ax2.set_ylabel(r"$\Delta$ SWE (mm)")
    ax2.set_xlabel('Year') 

In [ ]:
def zone_list(ch5_id):
    return [cz.zone for cz in target_zones if cz.ch5_id == ch5_id]

In [ ]:
[plot_zone(zone_list(ch5_id)) for ch5_id in target_ch5ids]